# Deep Dream

In [1]:
import os
import numpy as np
import torch
from torch.optim import SGD
from torchvision import models
import torch.nn as nn
from torch.autograd import Variable
import copy

import numpy as np
from PIL import Image
import matplotlib.cm as mpl_color_map
from matplotlib.colors import LogNorm
from astropy.io import fits
import matplotlib.pyplot as plt

from skimage.transform import resize
from scipy.ndimage import gaussian_filter
from CNN_Networks import OU200_CNN, VIS_CNN, JYH_CNN, OU66_CNN
from Generate_Class_Examples import ClassSpecificImageGeneration

## Load States
A dictionary containing all the weights for the CNNs used in the accompanning paper. 

In [2]:
loadStates = {'J': 'OU_J_Weights.pt',
              'Y': 'OU_Y_Weights.pt',
              'H': 'OU_H_Weights.pt',
              'JYH': 'OU_JYH_Weights.pt',
              'VIS': 'OU_VIS_Weights.pt',
              'OU-66': 'OU_66_Weights.pt',
              'OU-200': 'OU_200_Weights.pt'}

## Load Image
This function loads a specific image that fits the OU-200 input expectations.

In [5]:
def Get200Images(name):
    images = np.zeros((1,4,200,200))
    image_data1 = fits.getdata('/Volumes/Seagate Backup Plus Drive/Euclid 2 TrainData/EUC_J/imageEUC_J-'+str(name)+'.fits', ext=0)
    image_data1 = resize(image_data1, (200,200))
    J = (image_data1-np.amin(image_data1))/(np.amax(image_data1)-np.amin(image_data1))
    
    image_data1 = fits.getdata('/Volumes/Seagate Backup Plus Drive/Euclid 2 TrainData/EUC_Y/imageEUC_Y-'+str(name)+'.fits', ext=0)
    image_data1 = resize(image_data1, (200,200))
    Y = (image_data1-np.amin(image_data1))/(np.amax(image_data1)-np.amin(image_data1))
    
    image_data1 = fits.getdata('/Volumes/Seagate Backup Plus Drive/Euclid 2 TrainData/EUC_H/imageEUC_H-'+str(name)+'.fits', ext=0)
    image_data1 = resize(image_data1, (200,200))
    H = (image_data1-np.amin(image_data1))/(np.amax(image_data1)-np.amin(image_data1))
    
    image_data1 = fits.getdata('/Volumes/Seagate Backup Plus Drive/Euclid 2 TrainData/EUC_VIS/imageEUC_VIS-'+str(name)+'.fits', ext=0)
    #image_data1 = resize(image_data1, (200,200))
    VIS = (image_data1-np.amin(image_data1))/(np.amax(image_data1)-np.amin(image_data1))
    
    images[0,0] = J
    images[0,1] = Y
    images[0,2] = H
    images[0,3] = VIS
    
    return images

## Making Deep Dream Images
We first define a set of variables for this particluar run k (which is an identifying number for this run), lr (learning rate), range_Images (how many interations this process runs for), and rand_loc (location of the intial input image). 

The CNN is selected and loaded with the trained weights. An image from the dataset is loaded. The file locations for the output of this notebook are specificied depending on the target class. These input variables are given to the ClassSpecificImageGeneration() method and the input image is updated to activate the target class. This function saves the numpy file of the image and a single band image every 10 interations. This process is carried out for both target classes.

In [6]:
k = 252238
lr = 0.1
range_Images = 100

for j in range(2):
                target_class = j 
                model = OU200_CNN()
                criterion = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
                model.load_state_dict(torch.load(loadStates['OU-200'],map_location=torch.device('cpu')))
                                
                input_img =  Get200Images(k)
    
                if target_class == 0:
                        file_loc = 'Non_lenses'
                else:
                        file_loc = 'Lenses'
 
                loc = 'generated/OU200/'+str(range_Images)+'/'+file_loc+'_'+str(lr)+'_No_'+str(k)+'C_SameRandom4AllBands/'
                if not os.path.exists(loc):
                    os.makedirs(loc)
            
                pretrained_model = model
                csig = ClassSpecificImageGeneration(pretrained_model, target_class, lr, loc, input_img, range_Images)
                csig.generate()

Iteration: 1 Loss 7.00 Acc: [5.2501405e-06 9.9999475e-01]
Iteration: 2 Loss 2.29 Acc: [0.01641562 0.9835844 ]
Iteration: 3 Loss 0.40 Acc: [0.19029096 0.809709  ]
Iteration: 4 Loss -1.12 Acc: [0.84721136 0.15278864]
Iteration: 5 Loss -1.65 Acc: [0.8512214  0.14877859]
Iteration: 6 Loss -1.92 Acc: [0.86244    0.13756001]
Iteration: 7 Loss -2.95 Acc: [0.9328573  0.06714275]
Iteration: 8 Loss -4.51 Acc: [0.9989485  0.00105155]
Iteration: 9 Loss -4.64 Acc: [0.9980654  0.00193463]
Iteration: 10 Loss -7.14 Acc: [9.9998188e-01 1.8125236e-05]
Iteration: 11 Loss -8.80 Acc: [9.9999821e-01 1.8371269e-06]
Iteration: 12 Loss -10.01 Acc: [9.9999952e-01 4.7326128e-07]
Iteration: 13 Loss -11.45 Acc: [9.999999e-01 1.379383e-07]
Iteration: 14 Loss -12.33 Acc: [1.00000000e+00 1.37818335e-08]
Iteration: 15 Loss -13.68 Acc: [1.000000e+00 3.748254e-09]
Iteration: 16 Loss -15.36 Acc: [1.0000000e+00 1.1418035e-10]
Iteration: 17 Loss -16.28 Acc: [1.0000000e+00 1.0056719e-10]
Iteration: 18 Loss -17.06 Acc: [1.00

<Figure size 432x288 with 0 Axes>